In [89]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import itertools



def load_dfs( MAX_DATE = None , MIN_DATE = None):
    nba_teams = ['MIL', 'CHI', 'CHA', 'TOR', 'BOS', 'PHX', 'OKC', 'LAC', 'IND',
           'BKN', 'MIN', 'UTA', 'SAS', 'DAL', 'CLE', 'NYK', 'POR', 'HOU',
           'DEN', 'MEM', 'SAC', 'PHI',  'ATL', 'LAL', 
           'WAS', 'ORL', 'GSW', 'NOP', 
           'MIA', 
           'DET']
    nba_team_ids =[1610612749, 1610612766, 1610612738, 1610612746, 1610612754,
           1610612750, 1610612741, 1610612742, 1610612762, 1610612759,
           1610612739, 1610612752, 1610612761, 1610612760, 1610612757,
           1610612751, 1610612745, 1610612756, 1610612743, 1610612755,
           1610612737, 1610612763, 1610612764, 1610612744, 1610612740,
           1610612753, 1610612747, 1610612758, 1610612748, 1610612765]

   



    nba_teams_df = pd.DataFrame(nba_teams)
    dummy_nba_teams= pd.get_dummies(nba_team_ids, prefix ='team')


    df = pd.read_csv('large_boxscoretraditionalv2_df.csv')
    df=df[df['TEAM_ID'].isin(nba_team_ids)]
    df.drop_duplicates(subset = ['GAME_ID','PLAYER_ID'],inplace=True)
    df.dropna(subset=['MIN'],inplace=True)



    df_g = pd.read_csv('all_games.csv')
    df_g.drop_duplicates(subset =['TEAM_ID','GAME_ID'],inplace=True)

    df_g.loc[df_g['WL']=='W','IsWin']=1
    df_g.loc[df_g['WL']=='L','IsWin']=0
    df_g['IsHome'] = (df_g['MATCHUP'].str.contains('vs.')).astype(int)
    df_g['Opposing_team'] = df_g['MATCHUP'].apply( lambda x: x.split(' ')[2])

    df_g=df_g[df_g['TEAM_ID'].isin(nba_team_ids)]
    team_abv_to_id_mapping = df_g[['TEAM_ABBREVIATION','TEAM_ID']].drop_duplicates().set_index('TEAM_ABBREVIATION')['TEAM_ID'].to_dict()
    df_g['Opposing_team_ID'] = df_g['Opposing_team'].apply(lambda x: team_abv_to_id_mapping[x] if x in team_abv_to_id_mapping else None )

    df_g=df_g[df_g['Opposing_team_ID'].isin(nba_team_ids)]

    # df_g=df_g[['GAME_ID','TEAM_ID','GAME_DATE']].drop_duplicates().dropna()
    df_g['GAME_DATE'] = pd.to_datetime(df_g['GAME_DATE'])
    
    if MAX_DATE is not None:
        df_g = df_g[df_g['GAME_DATE']<MAX_DATE]
    if MIN_DATE is not None:
        df_g = df_g[df_g['GAME_DATE']>MIN_DATE]

    # add balance scoring


    df = df.merge(df_g, 
         how='inner',
         left_on=['GAME_ID','TEAM_ID'],
         right_on=['GAME_ID','TEAM_ID'])


    pbp = pd.read_csv('large_playbyplayv2_df.csv', index_col=0)
    pbp.drop_duplicates(subset=['GAME_ID','EVENTNUM','PERIOD'], inplace=True)


    shot_chart_df =  pd.read_csv('ShotChartDetail_v2.csv')
    shot_chart_df.drop_duplicates(subset=['GAME_ID','GAME_EVENT_ID'], inplace=True)
    
    
    df_g['num_of_possessions'] = df_g.apply( lambda x: get_possessions_per_game_and_team(x,pbp), axis=1)
    df_g.dropna(subset=['num_of_possessions'],inplace=True)
    df_g['AdjustedPM'] = (df_g['PLUS_MINUS']/df_g['num_of_possessions'])*100
    df_g['OffRating'] = (df_g['PTS']/df_g['num_of_possessions'])*100
    df_g['EFG'] = (df_g['FGM'] + 0.5*df_g['FG3M'])/df_g['FGA']
    df_g['AST_ratio'] = df_g['AST']*100/((df_g['FGA'])+(df_g['FTA']*0.44)+(df_g['AST'])+(df_g['AST']))
    df_g['Opp_points'] = df_g['PTS'] + df_g['PLUS_MINUS']
    df_g['Def_Rating'] = (df_g['Opp_points'] / df_g['num_of_possessions'])*100

    ['AdjustedPM','OffRating','EFG','AST_ratio','Opp_points','Def_Rating']



    print('done')
    return df,df_g,pbp,shot_chart_df,team_abv_to_id_mapping

df,df_g,pbp,shot_chart_df,team_abv_to_id_mapping = load_dfs(  MAX_DATE = '2020-01-01',MIN_DATE = '2019-01-01')

done


In [2]:
import numpy as np
def get_possessions_per_game_and_team(row,pbp):
    try:
        GAME_ID = row['GAME_ID']
        is_home = row['IsHome']
        single_game = pbp[pbp['GAME_ID']==GAME_ID]
        index_indicator = 'Home_EOP' if is_home==1 else 'Visitor_EOP'

        single_game = single_game.sort_values('EVENTNUM')
        single_game['Shift_HOMEDESCRIPTION'] = single_game['HOMEDESCRIPTION'].shift(-1)
        single_game[['Shift_HOMEDESCRIPTION','HOMEDESCRIPTION']]
        single_game.loc[(single_game['HOMEDESCRIPTION'].isnull()==False)&
                        (single_game['Shift_HOMEDESCRIPTION'].isnull()==True)&
                        (single_game['EVENTMSGTYPE'].isin([1,2,3,4,5,7,9])),'Home_EOP'] = 1
        single_game['Home_EOP'].fillna(0,inplace=True)
        single_game[['Shift_HOMEDESCRIPTION','HOMEDESCRIPTION','VISITORDESCRIPTION','Home_EOP']]

        single_game['Shift_VISITORDESCRIPTION'] = single_game['VISITORDESCRIPTION'].shift(-1)
        single_game[['Shift_VISITORDESCRIPTION','VISITORDESCRIPTION']]
        single_game.loc[(single_game['VISITORDESCRIPTION'].isnull()==False)&
                        (single_game['Shift_VISITORDESCRIPTION'].isnull()==True)&
                        (single_game['EVENTMSGTYPE'].isin([1,2,3,4,5,7,9])),'Visitor_EOP'] = 1
        single_game['Visitor_EOP'].fillna(0,inplace=True)
        return single_game[['Home_EOP','Visitor_EOP']].sum().loc[index_indicator]
    except:
        return np.nan




In [90]:
nba_teams = ['MIL', 'CHI', 'CHA', 'TOR', 'BOS', 'PHX', 'OKC', 'LAC', 'IND',
       'BKN', 'MIN', 'UTA', 'SAS', 'DAL', 'CLE', 'NYK', 'POR', 'HOU',
       'DEN', 'MEM', 'SAC', 'PHI',  'ATL', 'LAL', 
       'WAS', 'ORL', 'GSW', 'NOP', 
       'MIA', 
       'DET']
nba_team_ids =[1610612749, 1610612766, 1610612738, 1610612746, 1610612754,
       1610612750, 1610612741, 1610612742, 1610612762, 1610612759,
       1610612739, 1610612752, 1610612761, 1610612760, 1610612757,
       1610612751, 1610612745, 1610612756, 1610612743, 1610612755,
       1610612737, 1610612763, 1610612764, 1610612744, 1610612740,
       1610612753, 1610612747, 1610612758, 1610612748, 1610612765]





nba_teams_df = pd.DataFrame(nba_teams)
dummy_nba_teams= pd.get_dummies(nba_team_ids, prefix ='team')

sliding_window_num_of_games = 3

large_df_of_sequences = pd.DataFrame([])
 
# create the sliding window dataframe per team
for team in nba_team_ids:
    print(team)
    x = df_g[df_g['TEAM_ID'] == team].sort_values('GAME_DATE').reset_index(drop=True)
    
    
    dummy_curr_team = pd.get_dummies(x['TEAM_ID'], prefix ='team').reindex(columns = dummy_nba_teams.columns, fill_value=0)
    dummy_curr_team.columns = ['curr_' + col for col in dummy_curr_team.columns]
    team_indicator = dummy_curr_team.loc[0].values
    
    
    list_team_indicator = list(team_indicator) 
    for ix,row in x[sliding_window_num_of_games:].iterrows():
        tmp = x.loc[ix-sliding_window_num_of_games:ix]
        
        # add sequence of wins
        list_of_sequence_of_wins = list(tmp['IsWin'].values[:-1])
        new_cols = []
        for game in range(sliding_window_num_of_games):
                new_cols.insert(0,'{}_games_back_IsWin'.format(game + 1))
        df_of_sequence_of_wins = pd.DataFrame([list_of_sequence_of_wins],columns=new_cols)
        
        # get team indicator
        team_indicator = dummy_curr_team[0:1]
        
        # is at home
        list_of_sequence_of_at_home = list(tmp['IsHome'].values)
        new_cols = []
        for game in range(sliding_window_num_of_games+1):
                new_cols.insert(0,'{}_games_back_IsHome'.format(game + 1))
        df_of_sequence_of_at_home = pd.DataFrame([list_of_sequence_of_at_home],columns=new_cols)
        
        # get opp team indicator 
        opp_dummy_teams = pd.get_dummies(tmp['Opposing_team_ID'].astype(int), prefix ='team').reindex(columns = dummy_nba_teams.columns, fill_value=0)
        opp_dummy_teams.columns = ['opp_' + col for col in opp_dummy_teams.columns]
        appended_list_of_encoded_opp_teams = list(itertools.chain(*opp_dummy_teams.values))
        appended_df_of_encoded_opp_teams = pd.DataFrame([appended_list_of_encoded_opp_teams])
        
        
        # add averages of categories
        cols_to_calc = ['PTS','REB','AST','STL','BLK','TOV','PLUS_MINUS','FG3_PCT','FG_PCT','FT_PCT','num_of_possessions',
                       'AdjustedPM','OffRating','EFG','AST_ratio','Opp_points','Def_Rating'
                       ]
        averages_of_categories_curr_team = tmp[cols_to_calc][:-1].mean().values
        df_of_averages_of_categories_curr_team = pd.DataFrame([averages_of_categories_curr_team],columns =cols_to_calc)
        
        
        # add shot area avgs for curr team
        shot_index= ['Mid-Range', 'In The Paint (Non-RA)', 'Restricted Area',
       'Above the Break 3', 'Left Corner 3', 'Right Corner 3']

        shot_chart_df_for_tmp = shot_chart_df[(shot_chart_df['GAME_ID'].isin(tmp[:-1]['GAME_ID'].values))&
                                 (shot_chart_df['TEAM_ID']==team)].groupby(
                                    ['SHOT_ZONE_BASIC']).mean()['SHOT_MADE_FLAG'].reset_index()
        shot_chart_df_for_tmp_reindex = shot_chart_df_for_tmp[['SHOT_ZONE_BASIC','SHOT_MADE_FLAG']].set_index('SHOT_ZONE_BASIC').reindex(shot_index).fillna(0)
        shot_area_avgs = shot_chart_df_for_tmp_reindex.T.reset_index(drop=True)
        
        # add shot chart of opposing teams during window
        shot_chart_df_for_tmp = shot_chart_df[(shot_chart_df['GAME_ID'].isin(tmp[:-1]['GAME_ID'].values))&
                                 (shot_chart_df['TEAM_ID']!=team)].groupby(
                                    ['SHOT_ZONE_BASIC']).mean()['SHOT_MADE_FLAG'].reset_index()
        shot_chart_df_for_tmp_reindex = shot_chart_df_for_tmp[['SHOT_ZONE_BASIC','SHOT_MADE_FLAG']].set_index('SHOT_ZONE_BASIC').reindex(shot_index).fillna(0)
        shot_area_avgs_opp = shot_chart_df_for_tmp_reindex.T.reset_index(drop=True)
        
        # add sequence of scores
        list_of_sequence_of_wins = list(tmp['PTS'].values)
        new_cols = []
        for game in range(sliding_window_num_of_games):
                new_cols.insert(0,'{}_games_back_IsWin'.format(game + 1))
        new_cols.append('Y') 
        df_of_sequence_of_scores = pd.DataFrame([list_of_sequence_of_wins],columns=new_cols)
        
        
         # add averages of categories for past opponents
        list_of_opp_ids = list(tmp['Opposing_team_ID'].unique())

        x_opp = df_g[(df_g['TEAM_ID'].isin(list_of_opp_ids))&
                    (df_g['GAME_DATE'] < tmp['GAME_DATE'].max())&
                    (df_g['GAME_DATE'] >= tmp['GAME_DATE'].min())].sort_values('GAME_DATE').reset_index(drop=True)
        tmp_opp = x_opp[-sliding_window_num_of_games:ix]
        cols_to_calc = ['PTS','REB','AST','STL','BLK','TOV','PLUS_MINUS','FG3_PCT','FG_PCT','FT_PCT','num_of_possessions',
                       'AdjustedPM','OffRating','EFG','AST_ratio','Opp_points','Def_Rating']
        averages_of_categories_opp_team = tmp_opp[cols_to_calc].mean().values
        df_of_averages_of_categories_opp_team = pd.DataFrame([averages_of_categories_opp_team],columns = cols_to_calc).reset_index(drop=True)
        
        # add sequence of days between games
        tmp['GAME_DATE'].diff().dt.days.dropna()
        l = tmp['GAME_DATE'].diff().dt.days.dropna().values
        l = [4 if sl >4 else sl for sl in l]
        new_cols = []
        for game in range(sliding_window_num_of_games):
                new_cols.insert(0,'{}_games_back_PTS'.format(game + 1))

        df_days_between = pd.DataFrame([l],columns=new_cols)

        # start merging all together 
        # 1. wins
        # 2. team indicator
        # 3. at home indicator
        # 4. TOV sequence
#         print(len(team_indicator.columns),'team_indicator')
#         print(len(df_of_sequence_of_wins.columns),'df_of_sequence_of_wins')
#         print(len(df_of_sequence_of_at_home.columns),'df_of_sequence_of_at_home')
#         print(len(appended_df_of_encoded_opp_teams.columns),'appended_df_of_encoded_opp_teams')
#         print(len(df_of_averages_of_categories_curr_team.columns),'df_of_averages_of_categories_curr_team')
#         print(len(shot_area_avgs.columns),'shot_area_avgs')
#         print(len(shot_area_avgs_opp.columns),'shot_area_avgs_opp')
        
        df_all_features = team_indicator.merge(df_of_sequence_of_wins , how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(df_of_sequence_of_at_home, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(appended_df_of_encoded_opp_teams, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(df_of_averages_of_categories_curr_team, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(shot_area_avgs, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(shot_area_avgs_opp, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(df_of_sequence_of_scores, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(df_of_averages_of_categories_opp_team, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(df_days_between, how='inner' , left_index=True , right_index=True)

        large_df_of_sequences = large_df_of_sequences.append(df_all_features)
    
large_df_of_sequences.dropna(inplace=True)



1610612749
1610612766
1610612738
1610612746
1610612754
1610612750
1610612741
1610612742
1610612762
1610612759
1610612739
1610612752
1610612761
1610612760
1610612757
1610612751
1610612745
1610612756
1610612743
1610612755
1610612737
1610612763
1610612764
1610612744
1610612740
1610612753
1610612747
1610612758
1610612748
1610612765


In [83]:
X

,curr_team_1610612737,curr_team_1610612738,curr_team_1610612739,curr_team_1610612740,curr_team_1610612741,curr_team_1610612742,curr_team_1610612743,curr_team_1610612744,curr_team_1610612745,curr_team_1610612746,...,AdjustedPM_y,OffRating_y,EFG_y,AST_ratio_y,Opp_points_y,Def_Rating_y,4_games_back_PTS,3_games_back_PTS,2_games_back_PTS,1_games_back_PTS
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.456161,0.413433,0.458723,0.554884,0.424342,0.371283,0.333333,0.333333,0.333333,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.611771,0.824498,0.842957,0.777355,0.666667,0.701010,0.333333,0.333333,0.000000,0.333333
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.357516,0.181365,0.400297,0.489291,0.171053,0.178161,0.333333,0.000000,0.333333,0.333333
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.662069,0.549979,0.646872,0.676656,0.582237,0.580420,0.000000,0.333333,0.333333,0.333333
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.455774,0.320807,0.389532,0.491442,0.296053,0.319382,0.333333,0.333333,0.333333,0.333333
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.502240,0.551093,0.686864,0.609954,0.416667,0.477807,0.333333,0.333333,0.333333,0.000000
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.334959,0.376524,0.366420,0.026316,0.032887,0.333333,0.333333,0.000000,0.333333
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.345709,0.357090,0.379020,0.431676,0.223684,0.268523,0.333333,0.000000,0.333333,0.333333
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.432994,0.437507,0.501776,0.679151,0.315789,0.369743,0.000000,0.333333,0.333333,0.333333
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.572968,0.410585,0.381131,0.474635,0.486842,0.445141,0.333333,0.333333,0.333333,0.666667


In [91]:
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn import preprocessing

#scale the X variables
X = large_df_of_sequences.loc[:, large_df_of_sequences.columns != 'Y']
saved_columns = X.columns
x = X.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X = pd.DataFrame(x_scaled, columns=saved_columns)


Y = large_df_of_sequences['Y']

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)

rf = RandomForestRegressor(n_estimators=1000, random_state=1)
rf.fit(x_train,y_train)


# Use the forest's predict method on the test data
predictions = rf.predict(x_test)
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')


# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')
    
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 4)) for feature, importance in zip(X.columns, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];    

y_predict = rf.predict(x_test)
# print ("R2 score:",r2_score(y_test,y_predict))


Mean Absolute Error: 9.41 degrees.
Accuracy: 91.26 %.
Variable: PTS_x                Importance: 0.0374
Variable: FG3_PCT_y            Importance: 0.027
Variable: Above the Break 3_x  Importance: 0.0242
Variable: FG3_PCT_x            Importance: 0.024
Variable: FT_PCT_x             Importance: 0.0238
Variable: Above the Break 3_y  Importance: 0.0221
Variable: In The Paint (Non-RA)_y Importance: 0.022
Variable: FT_PCT_y             Importance: 0.022
Variable: REB_x                Importance: 0.0213
Variable: Restricted Area_y    Importance: 0.0207
Variable: Mid-Range_y          Importance: 0.02
Variable: 2_games_back_IsWin_y Importance: 0.02
Variable: Mid-Range_x          Importance: 0.0199
Variable: Restricted Area_x    Importance: 0.0188
Variable: STL_y                Importance: 0.0182
Variable: In The Paint (Non-RA)_x Importance: 0.0181
Variable: OffRating_x          Importance: 0.0174
Variable: 1_games_back_IsWin_y Importance: 0.0172
Variable: 3_games_back_IsWin_y Importance: 0.016

In [92]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier , RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Create param grid.

param_grid = {
    'bootstrap': [True,False],
    'max_depth': [110,150],
    'max_features': [20,200],
    'min_samples_leaf': [3],
    'min_samples_split': [ 12],
    'n_estimators': [  500]
}
rf = RandomForestRegressor()

# Create grid search object

clf = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

# Fit on data

best_clf = clf.fit(x_train, y_train)
from sklearn.metrics import classification_report
print("Best parameters set found on development set:")
print(best_clf.best_params_)
print("Detailed classification report:")
y_true, y_pred = y_test, best_clf.predict(x_test)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   52.5s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:  2.4min remaining:    7.4s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  2.4min finished


Best parameters set found on development set:
{'bootstrap': False, 'max_depth': 150, 'max_features': 20, 'min_samples_leaf': 3, 'min_samples_split': 12, 'n_estimators': 500}
Detailed classification report:


In [12]:
from sklearn.metrics import r2_score

def RFR(X_train, X_test, y_train, y_test, best_params):
    from sklearn.ensemble import RandomForestRegressor
    estimator = RandomForestRegressor(n_jobs=-1).set_params(**best_params)
    estimator.fit(X_train,y_train)
    y_predict = estimator.predict(X_test)
    print ("R2 score:",r2_score(y_test,y_predict))
    return y_test,y_predict


# best_score, best_params = Grid_Search_CV_RFR(X_train, y_train)
y_test , y_predict = RFR(x_train, x_test, y_train, y_test ,best_clf.best_params_ )
print ("Best Score:" ,best_clf.best_score_)


NameError: name 'best_clf' is not defined

In [94]:
import statsmodels.api as sm
X_cons = sm.add_constant(X)
reg_model = sm.OLS(Y, X_cons.values).fit()
predictions = reg_model.predict(X_cons) 

print_model = reg_model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.186
Method:                 Least Squares   F-statistic:                     3.549
Date:                Mon, 27 Jan 2020   Prob (F-statistic):           7.11e-47
Time:                        09:43:51   Log-Likelihood:                -8412.1
No. Observations:                2215   AIC:                         1.722e+04
Df Residuals:                    2015   BIC:                         1.836e+04
Df Model:                         199                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         88.8717      8.296     10.713      0.0

In [98]:

def predict_game_score(
                        curr_team_abv,
                        opp_team,
                        is_current_game_at_home,
                        curr_team_id,
                        df_g,
                        df,
                        pbp,
                        shot_chart_df):
    
    
    
    
    x = df_g[df_g['TEAM_ID'] == curr_team_id].sort_values('GAME_DATE').reset_index(drop=True)
    tmp = x[-sliding_window_num_of_games:]

    dummy_curr_team = pd.get_dummies(x['TEAM_ID'], prefix ='team').reindex(columns = dummy_nba_teams.columns, fill_value=0)
    dummy_curr_team.columns = ['curr_' + col for col in dummy_curr_team.columns]
    team_indicator = dummy_curr_team.loc[0].values

    list_of_sequence_of_wins = list(tmp['IsWin'].values)
    new_cols = []
    for game in range(sliding_window_num_of_games):
            new_cols.insert(0,'{}_games_back_IsWin'.format(game + 1))
    df_of_sequence_of_wins = pd.DataFrame([list_of_sequence_of_wins],columns=new_cols)

    # # get team indicator
    team_indicator = dummy_curr_team[0:1]
    team_indicator

    # is at home
    list_of_sequence_of_at_home = list(tmp['IsHome'].values)
    new_cols = []
    for game in range(sliding_window_num_of_games):
            new_cols.insert(0,'{}_games_back_IsHome'.format(game + 1))
    df_of_sequence_of_at_home = pd.DataFrame([list_of_sequence_of_at_home],columns=new_cols)
    df_of_sequence_of_at_home['{}_games_back_IsHome'.format(sliding_window_num_of_games+1)] = is_current_game_at_home


    # get opp team indicator 

    opp_dummy_teams = pd.get_dummies(tmp['Opposing_team_ID'].astype(int).append(pd.Series(opp_team_id)), prefix ='team').reindex(columns = dummy_nba_teams.columns, fill_value=0)
    opp_dummy_teams.columns = ['opp_' + col for col in opp_dummy_teams.columns]
    appended_list_of_encoded_opp_teams = list(itertools.chain(*opp_dummy_teams.values))
    appended_df_of_encoded_opp_teams = pd.DataFrame([appended_list_of_encoded_opp_teams])

    # add averages of categories
    cols_to_calc = ['PTS','REB','AST','STL','BLK','TOV','PLUS_MINUS','FG3_PCT','FG_PCT','FT_PCT','num_of_possessions',
                       'AdjustedPM','OffRating','EFG','AST_ratio','Opp_points','Def_Rating']
    averages_of_categories_curr_team = tmp[cols_to_calc].mean().values
    df_of_averages_of_categories_curr_team = pd.DataFrame([averages_of_categories_curr_team],columns =cols_to_calc)
    df_of_averages_of_categories_curr_team

    # add shot area avgs for curr team
    shot_index= ['Mid-Range', 'In The Paint (Non-RA)', 'Restricted Area',
    'Above the Break 3', 'Left Corner 3', 'Right Corner 3']

    shot_chart_df_for_tmp = shot_chart_df[(shot_chart_df['GAME_ID'].isin(tmp['GAME_ID'].values))&
                             (shot_chart_df['TEAM_ID']==curr_team_id)].groupby(
                                ['SHOT_ZONE_BASIC']).mean()['SHOT_MADE_FLAG'].reset_index()
    shot_chart_df_for_tmp_reindex = shot_chart_df_for_tmp[['SHOT_ZONE_BASIC','SHOT_MADE_FLAG']].set_index('SHOT_ZONE_BASIC').reindex(shot_index).fillna(0)
    shot_area_avgs = shot_chart_df_for_tmp_reindex.T.reset_index(drop=True)
    shot_area_avgs

    # add shot chart of opposing teams during window
    shot_chart_df_for_tmp = shot_chart_df[(shot_chart_df['GAME_ID'].isin(tmp[:-1]['GAME_ID'].values))&
                             (shot_chart_df['TEAM_ID']!=curr_team_id)].groupby(
                                ['SHOT_ZONE_BASIC']).mean()['SHOT_MADE_FLAG'].reset_index()
    shot_chart_df_for_tmp_reindex = shot_chart_df_for_tmp[['SHOT_ZONE_BASIC','SHOT_MADE_FLAG']].set_index('SHOT_ZONE_BASIC').reindex(shot_index).fillna(0)
    shot_area_avgs_opp = shot_chart_df_for_tmp_reindex.T.reset_index(drop=True)
    shot_area_avgs_opp
    
    
    # add sequence of scores
    list_of_sequence_of_wins = list(tmp['PTS'].values)
    new_cols = []
    for game in range(sliding_window_num_of_games):
            new_cols.insert(0,'{}_games_back_IsWin'.format(game + 1))
    df_of_sequence_of_scores = pd.DataFrame([list_of_sequence_of_wins],columns=new_cols)

        
     # add averages of categories
    x_opp = df_g[df_g['TEAM_ID'] == opp_team_id].sort_values('GAME_DATE').reset_index(drop=True)
    tmp_opp = x_opp[-sliding_window_num_of_games:]
    cols_to_calc = ['PTS','REB','AST','STL','BLK','TOV','PLUS_MINUS','FG3_PCT','FG_PCT','FT_PCT','num_of_possessions',
                       'AdjustedPM','OffRating','EFG','AST_ratio','Opp_points','Def_Rating']
    averages_of_categories_opp_team = tmp_opp[cols_to_calc].mean().values
    df_of_averages_of_categories_opp_team = pd.DataFrame([averages_of_categories_opp_team],columns =cols_to_calc)
    df_of_averages_of_categories_opp_team
    
    # add sequence of days between games
    tmp_days_back = x[-sliding_window_num_of_games-1:]
    l = tmp_days_back['GAME_DATE'].diff().dt.days.dropna().values
    l = [4 if sl >4 else sl for sl in l]
    new_cols = []
    for game in range(sliding_window_num_of_games):
            new_cols.insert(0,'{}_games_back_PTS'.format(game + 1))
    df_days_between = pd.DataFrame([l],columns=new_cols)

    # start merging all together 
    # 1. wins
    # 2. team indicator
    # 3. at home indicator
    # 4. TOV sequence
    # print(len(team_indicator.columns),'team_indicator')
    # print(len(df_of_sequence_of_wins.columns),'df_of_sequence_of_wins')
    # print(len(df_of_sequence_of_at_home.columns),'df_of_sequence_of_at_home')
    # print(len(appended_df_of_encoded_opp_teams.columns),'appended_df_of_encoded_opp_teams')
    # print(len(df_of_averages_of_categories_curr_team.columns),'df_of_averages_of_categories_curr_team')
    # print(len(shot_area_avgs.columns),'shot_area_avgs')
    # print(len(shot_area_avgs_opp.columns),'shot_area_avgs_opp')
    df_all_features = team_indicator.merge(df_of_sequence_of_wins , how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(df_of_sequence_of_at_home, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(appended_df_of_encoded_opp_teams, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(df_of_averages_of_categories_curr_team, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(shot_area_avgs, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(shot_area_avgs_opp, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(df_of_sequence_of_scores, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(df_of_averages_of_categories_opp_team, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(df_days_between, how='inner' , left_index=True , right_index=True)

    len(df_all_features.columns)
    
#     len(X.columns)

#     print([curr_team_abv,'------->' , best_clf.predict(df_all_features) , '-------',
#                         opp_team,
#                         is_current_game_at_home,
#                         curr_team_id])
#     print()
    df_all_features = df_all_features.loc[:, df_all_features.columns != 'Y']
    df_all_features = pd.DataFrame(min_max_scaler.transform(df_all_features),columns=df_all_features.columns)
    return df_all_features, best_clf.predict(df_all_features)

In [96]:
# df_all_features = df_all_features.loc[:, df_all_features.columns != 'Y']
# pd.DataFrame(min_max_scaler.transform(df_all_features),columns=df_all_features.columns)


In [97]:
# lsit_of_games = [
#                 ['TOR','POR'] , 
#                 ['CLE','DET'] , 
#                 ['BKN','OKC'] , 
#                 ['MEM','MIN'] , 
#                 ['PHX','SAC'] , 
#                 ['LAL','NYK'] , 
                
#             ]

# df,df_g,pbp,shot_chart_df,team_abv_to_id_mapping = load_dfs( MAX_DATE = '2020-01-07')

total_results_saved = []


LAST_DATE_TO_RUN_ON = '2020-01-27'
FIRST_DATE_TO_CHECK = '2020-01-01'

df,df_g_global ,pbp,shot_chart_df,team_abv_to_id_mapping = load_dfs( MAX_DATE = LAST_DATE_TO_RUN_ON)

list_of_games_to_check = df_g_global[['GAME_DATE','TEAM_ABBREVIATION','GAME_ID','IsHome']]
len(list_of_games_to_check)
merged_list_of_games_to_check = list_of_games_to_check.merge(list_of_games_to_check,
                                                             how='inner',
                                                             left_on =['GAME_ID'] ,
                                                             right_on =['GAME_ID'])
merged_list_of_games_to_check = merged_list_of_games_to_check[merged_list_of_games_to_check['TEAM_ABBREVIATION_x']!= merged_list_of_games_to_check['TEAM_ABBREVIATION_y']]
merged_list_of_games_to_check=merged_list_of_games_to_check[merged_list_of_games_to_check['IsHome_x']==1]


merged_list_of_games_to_check = merged_list_of_games_to_check[merged_list_of_games_to_check['GAME_DATE_x']>FIRST_DATE_TO_CHECK]
list_of_dates = merged_list_of_games_to_check['GAME_DATE_x'].unique()
for single_date in list_of_dates:
    print (single_date)
    lsit_of_games = []
    for ix,row in merged_list_of_games_to_check[merged_list_of_games_to_check['GAME_DATE_x']==single_date].iterrows():
        game=[row['TEAM_ABBREVIATION_x'],row['TEAM_ABBREVIATION_y']]
        lsit_of_games.append(game)
    df,df_g,pbp,shot_chart_df,team_abv_to_id_mapping = load_dfs( MAX_DATE = str(single_date)[:10])
     

    sliding_window_num_of_games = 3

    for game in lsit_of_games:
    #     print(df_g['GAME_DATE'].max())
    #     df_g['TEAM_ABBREVIATION'].unique()
        print(game)
        print(df_g['GAME_DATE'].max())
        df_g['TEAM_ABBREVIATION'].unique()
        ###########################
        ### DEFINE YOUR TEAMS   ###
        ###########################
        curr_team_abv = game[0] #'PHX'
        curr_team_id = team_abv_to_id_mapping[curr_team_abv]

        opp_team = game[1] #'SAC'
        opp_team_id  = team_abv_to_id_mapping[opp_team]
        is_current_game_at_home = 1



        ###########################


        res_1,rf_score_1 = predict_game_score(
                            curr_team_abv,
                           opp_team,
                           1,
                           curr_team_id ,
                          df_g,
                          df,
                          pbp,
                          shot_chart_df)
    #     res = sm.add_constant(res)
        res_1['const'] = 1
        reg_score_1 = reg_model.predict(res_1[X_cons.columns])
    #     print('RES' , reg_model.predict(res_1[X_cons.columns]))
        res_2 , rf_score_2 = predict_game_score(
                            opp_team,
                           curr_team_abv,
                           0,
                           opp_team_id,
                          df_g,
                          df,
                          pbp,
                          shot_chart_df)
    #     res = sm.add_constant(res)
        res_2['const'] = 1
        reg_score_2 =  reg_model.predict(res_2[X_cons.columns])

        print('RF:' , rf_score_1[0],':',rf_score_2[0] )
        print ('LR:' , reg_score_1.values[0],':',reg_score_2.values[0] )
        print()
        results_saved = [single_date,curr_team_abv,opp_team,
                        rf_score_1[0],rf_score_2[0],
                         reg_score_1.values[0],reg_score_2.values[0]]
        total_results_saved.append(results_saved)

done
2020-01-06T00:00:00.000000000
done
['CHA', 'IND']
2020-01-05 00:00:00
RF: 113.65314747474754 : 115.61401406926403
LR: -1333.2657973000487 : -1358.92019510537

['ORL', 'BKN']
2020-01-05 00:00:00
RF: 116.66239581529584 : 115.84499249639252
LR: -1132.888523292681 : -1217.7392654500259

['ATL', 'DEN']
2020-01-05 00:00:00
RF: 116.72082979797976 : 115.28628463203462
LR: -1239.4002845709456 : -1246.2315842649532

['DAL', 'CHI']
2020-01-05 00:00:00
RF: 117.08988152958152 : 114.9963112554113
LR: -1488.1163926268532 : -1137.0308254650304

['WAS', 'BOS']
2020-01-05 00:00:00
RF: 114.834061904762 : 116.22221832611831
LR: -1279.7042995247284 : -1327.071525186749

['NOP', 'UTA']
2020-01-05 00:00:00
RF: 116.69174365079363 : 116.02385808080804
LR: -1540.91167199628 : -1431.2888874228925

['PHI', 'OKC']
2020-01-05 00:00:00
RF: 116.3863443722944 : 116.55703051948053
LR: -1174.4363152481633 : -1387.494476160026

['SAS', 'MIL']
2020-01-05 00:00:00
RF: 116.27140447330457 : 116.16014264069264
LR: -1137.

RF: 119.13342417027422 : 119.24869574314582
LR: -1164.9282159934346 : -1247.9131591049381

['LAC', 'GSW']
2020-01-09 00:00:00
RF: 115.13727597402602 : 115.25122712842712
LR: -1620.018681061235 : -1143.720274326898

['UTA', 'CHA']
2020-01-09 00:00:00
RF: 116.52265295815297 : 114.14427308802311
LR: -1508.8376421136504 : -1206.6011090347843

['CHI', 'IND']
2020-01-09 00:00:00
RF: 114.02080158730163 : 114.35190505050512
LR: -1094.0852954368097 : -1310.93349692417

['NYK', 'NOP']
2020-01-09 00:00:00
RF: 115.7526867965369 : 116.47417813852815
LR: -1105.6747702095258 : -1364.5866705003773

['MEM', 'SAS']
2020-01-09 00:00:00
RF: 116.55765367965364 : 115.28161919191918
LR: -1377.0470997761468 : -1374.8554252324361

['SAC', 'MIL']
2020-01-09 00:00:00
RF: 116.77077662337668 : 116.23599141414141
LR: -1431.8451840290036 : -1325.5035203283655

['PHX', 'ORL']
2020-01-09 00:00:00
RF: 111.19700209235207 : 112.30772582972587
LR: -1395.1318594143313 : -1344.8500444871943

2020-01-09T00:00:00.000000000
do

RF: 112.87536940836938 : 111.78982864357863
LR: -1672.2885137535832 : -1408.6413820246244

['SAC', 'DAL']
2020-01-14 00:00:00
RF: 114.577821933622 : 116.75895360750361
LR: -1186.485194263079 : -1391.9375701763202

2020-01-21T00:00:00.000000000
done
['DAL', 'LAC']
2020-01-20 00:00:00
RF: 115.80416370851373 : 116.09967063492061
LR: -1468.4291059228426 : -1485.1742192675456

2020-01-20T00:00:00.000000000
done
['CLE', 'NYK']
2020-01-19 00:00:00
RF: 115.1018593795094 : 115.0084533910534
LR: -1338.9771152818544 : -1144.9677079862304

['BOS', 'LAL']
2020-01-19 00:00:00
RF: 115.61851320346324 : 115.87122431457432
LR: -1309.2703460026678 : -1353.5433505818369

['PHX', 'SAS']
2020-01-19 00:00:00
RF: 117.51356760461765 : 116.24635959595966
LR: -1519.6991162194076 : -1259.7287334804087

['BKN', 'PHI']
2020-01-19 00:00:00
RF: 115.2526430735932 : 116.3521803751804
LR: -1299.267568852621 : -1235.0403043117021

['MIA', 'SAC']
2020-01-19 00:00:00
RF: 116.26804220779219 : 114.4555960317461
LR: -1258.969

In [88]:
pd.DataFrame(total_results_saved
            ,columns=['Date','HomeTeam','AwayTeam','RFHome','RFAway','LRHome','LRAway']).to_csv('~/RF_LR_Jan.csv',index=False)
# .to_csv('~/probs_Jan.csv',index=False)
total_results_saved

[[numpy.datetime64('2020-01-06T00:00:00.000000000'),
  'CHA',
  'IND',
  112.90963622151804,
  113.22627218934315,
  559.5598453981693,
  620.2576476073136],
 [numpy.datetime64('2020-01-06T00:00:00.000000000'),
  'ORL',
  'BKN',
  114.2901703781427,
  112.90884927117078,
  687.5627569582741,
  298.74623813688504],
 [numpy.datetime64('2020-01-06T00:00:00.000000000'),
  'ATL',
  'DEN',
  113.12809846544059,
  113.5080915978328,
  888.9237747926386,
  505.125022186601],
 [numpy.datetime64('2020-01-06T00:00:00.000000000'),
  'DAL',
  'CHI',
  113.82054120044035,
  113.40902091943997,
  621.5321128852636,
  1066.3382358225049],
 [numpy.datetime64('2020-01-06T00:00:00.000000000'),
  'WAS',
  'BOS',
  113.7986040699418,
  114.12866036986824,
  388.2389609468985,
  654.4091829738957],
 [numpy.datetime64('2020-01-06T00:00:00.000000000'),
  'NOP',
  'UTA',
  113.88711544143072,
  114.4899587886923,
  136.02849702818457,
  631.1429876228499],
 [numpy.datetime64('2020-01-06T00:00:00.000000000'),
 

In [11]:
################
# RUN PREDICTED
################
LAST_DATE_TO_RUN_ON = '2020-01-22'

df,df_g_global ,pbp,shot_chart_df,team_abv_to_id_mapping = load_dfs( MAX_DATE = LAST_DATE_TO_RUN_ON)
           
lsit_of_games = [
                ['DET','SAC'] , 
                ['ORL','OKC'] , 
                ['MIA','WAS'] , 
                ['NYK','LAL'] , 
                ['TOR','PHI'] , 
                ['ATL','LAC'] , 
                ['BOS','MEM'] , 
                ['NOP','SAS'] , 
                ['HOU','DEN'] , 
                ['CHI','MIN'] , 
                ['PHX','IND'] , 
                ['GSW','UTA'] , 
    
                
            ]


        

sliding_window_num_of_games = 4

for game in lsit_of_games:
    print(game)
#     print(df_g['GAME_DATE'].max())
#     df_g['TEAM_ABBREVIATION'].unique()
    print(game)
    print(df_g_global['GAME_DATE'].max())
    df_g['TEAM_ABBREVIATION'].unique()
    ###########################
    ### DEFINE YOUR TEAMS   ###
    ###########################
    curr_team_abv = game[0] #'PHX'
    curr_team_id = team_abv_to_id_mapping[curr_team_abv]

    opp_team = game[1] #'SAC'
    opp_team_id  = team_abv_to_id_mapping[opp_team]
    is_current_game_at_home = 1



    ###########################


    res_1,rf_score_1 = predict_game_score(
                            curr_team_abv,
                           opp_team,
                           1,
                           curr_team_id ,
                          df_g,
                          df,
                          pbp,
                          shot_chart_df)
    #     res = sm.add_constant(res)
    res_1['const'] = 1
    reg_score_1 = reg_model.predict(res_1[X_cons.columns])
#     print('RES' , reg_model.predict(res_1[X_cons.columns]))
    res_2 , rf_score_2 = predict_game_score(
                        opp_team,
                       curr_team_abv,
                       0,
                       opp_team_id,
                      df_g,
                      df,
                      pbp,
                      shot_chart_df)
#     res = sm.add_constant(res)
    res_2['const'] = 1
    reg_score_2 =  reg_model.predict(res_2[X_cons.columns])

    print('RF:' , rf_score_1[0],':',rf_score_2[0] )
    print ('LR:' , reg_score_1.values[0],':',reg_score_2.values[0] )
    print()

done
['DET', 'SAC']
['DET', 'SAC']
2020-01-21 00:00:00
RF: 112.25169314574323 : 108.3506965367966
LR: 103.0834368794563 : 108.59828867341098

['ORL', 'OKC']
['ORL', 'OKC']
2020-01-21 00:00:00
RF: 107.70154022176663 : 111.53435147437104
LR: 104.3739657065652 : 109.21666296639424

['MIA', 'WAS']
['MIA', 'WAS']
2020-01-21 00:00:00
RF: 117.02999646464649 : 113.69410981240988
LR: 115.33870542275908 : 114.998646068146

['NYK', 'LAL']
['NYK', 'LAL']
2020-01-21 00:00:00
RF: 106.8282418529837 : 115.54050645272599
LR: 102.08979511012969 : 114.65775180024661

['TOR', 'PHI']
['TOR', 'PHI']
2020-01-21 00:00:00
RF: 114.30862950937961 : 107.6934651570652
LR: 116.19421627813769 : 109.79117395453575

['ATL', 'LAC']
['ATL', 'LAC']
2020-01-21 00:00:00
RF: 110.2402968128363 : 115.42101313131309
LR: 117.16614309055093 : 119.83466783423827

['BOS', 'MEM']
['BOS', 'MEM']
2020-01-21 00:00:00
RF: 113.30433762155718 : 116.70907581453645
LR: 113.95209959147456 : 115.33850122816585

['NOP', 'SAS']
['NOP', 'SAS']
